In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

from nltk.cluster import KMeansClusterer, cosine_distance

import json
from numpy.random import shuffle

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


from sklearn.decomposition import LatentDirichletAllocation

from scipy.cluster.hierarchy import cut_tree

from sklearn import metrics
import numpy as np




In [4]:
data=json.load(open('ydata_3group.json','r'))


shuffle(data)

text,target,nr=zip(*data)
text=list(text)
target=list(target)



data=[item for item in data if item[1] in \
      ['T1','T2', 'T3']]
tfidf_vect = TfidfVectorizer(stop_words="english",\
                             min_df=5) 

# generate tfidf matrix
dtm= tfidf_vect.fit_transform(text)
print (dtm.shape)

num_clusters=3
clusterer = KMeansClusterer(num_clusters, \
                            cosine_distance, repeats=10)
clusters = clusterer.cluster(dtm.toarray(), \
                            assign_clusters=True)

print(clusters[0:5])

df=pd.DataFrame(list(zip(target, clusters)), \
                columns=['actual_class','cluster'])
df.head()
pd.crosstab( index=df.cluster, columns=df.actual_class)


cluster_dict={0:'T3', 1:"T1",\
              2:'T2'}

# Assign true class to cluster
predicted_target=[cluster_dict[i] for i in clusters]

print(metrics.classification_report\
      (target, predicted_target))
centroids=np.array(clusterer.means())

sorted_centroids = centroids.argsort()[:, ::-1] 

voc_lookup= tfidf_vect.get_feature_names()

for i in range(num_clusters):
    
    # get words with top 5 tf-idf weight in the centroid
    top_words=[voc_lookup[word_index] \
               for word_index in sorted_centroids[i, :5]]
    print("Cluster %d: %s " % (i, "; ".join(top_words)))







(4021, 14959)
[2, 1, 0, 1, 0]


,actual_class,cluster
0,T2,2
1,T1,1
2,T1,0
3,T1,1
4,T1,0


actual_class,T1,T2,T3
cluster,,,
0,187,28,780
1,1215,49,368
2,66,1267,61


             precision    recall  f1-score   support

         T1       0.74      0.83      0.78      1468
         T2       0.91      0.94      0.93      1344
         T3       0.78      0.65      0.71      1209

avg / total       0.81      0.81      0.81      4021

Cluster 0: rail; plane; said; train; airlines 
Cluster 1: said; people; oil; police; bp 
Cluster 2: percent; tax; said; year; economy 


In [4]:

tf_vectorizer = CountVectorizer(max_df=0.90, \
                min_df=50, stop_words='english')
tf = tf_vectorizer.fit_transform(text)
tf_feature_names = tf_vectorizer.get_feature_names()
print(tf_feature_names[0:10])
print(tf.shape)
num_topics = 3
lda = LatentDirichletAllocation(n_components=num_topics, \
                                max_iter=10,verbose=1,
                                evaluate_every=1, n_jobs=1,
                                random_state=0).fit(tf)
num_top_words=5
for topic_idx, topic in enumerate(lda.components_):
    print ("Topic %d:" % (topic_idx))
    # print out top 20 words per topic 
    words=[(tf_feature_names[i],topic[i]) for i in topic.argsort()[::-1][0:num_top_words]]
    print(words)
    print("\n")
topic_assign=lda.transform(tf)
print(topic_assign[0:5])
topics=np.copy(topic_assign)
y=np.argsort(topics)
a=y[:,::-1]
a=[(row[1][0]) \
          for row in enumerate(a)]
df=pd.DataFrame(list(zip(target, a)), \
                columns=['actual_class','Topic'])
df.head()
pd.crosstab( index=df.Topic, columns=df.actual_class)
cluster_dict={0:'T3', 1:"T1",\
              2:'T2'}
predicted_target=[cluster_dict[i] for i in a]
print(metrics.classification_report\
      (target, predicted_target))

['abandoned', 'abc', 'ability', 'able', 'aboard', 'abroad', 'absolutely', 'abuse', 'accept', 'access']
(4021, 3209)


C:\Users\rajpu\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


iteration: 1 of max_iter: 10, perplexity: 1544.2245
iteration: 2 of max_iter: 10, perplexity: 1522.0037
iteration: 3 of max_iter: 10, perplexity: 1515.2717
iteration: 4 of max_iter: 10, perplexity: 1511.6576
iteration: 5 of max_iter: 10, perplexity: 1509.0786
iteration: 6 of max_iter: 10, perplexity: 1507.0276
iteration: 7 of max_iter: 10, perplexity: 1505.2967
iteration: 8 of max_iter: 10, perplexity: 1503.8241
iteration: 9 of max_iter: 10, perplexity: 1502.6148
iteration: 10 of max_iter: 10, perplexity: 1501.6717
Topic 0:
[('said', 5846.5416212662058), ('percent', 4592.7115597711927), ('year', 3706.4649260805468), ('tax', 3462.3799680972952), ('government', 2674.3904915590301)]


Topic 1:
[('said', 7593.7203745040897), ('comment', 3880.5919396687464), ('people', 3540.7863729517485), ('news', 2221.0997802385077), ('oil', 2069.602146212721)]


Topic 2:
[('said', 4425.4533033790485), ('new', 2108.6128101371378), ('com', 1632.3103834623078), ('rail', 1453.3773591058691), ('service', 1317

,actual_class,Topic
0,T1,1
1,T2,0
2,T1,1
3,T2,0
4,T1,1


actual_class,T1,T2,T3
Topic,,,
0,51,1216,79
1,988,40,63
2,429,88,1067


             precision    recall  f1-score   support

         T1       0.91      0.67      0.77      1468
         T2       0.06      0.07      0.06      1344
         T3       0.06      0.07      0.06      1209

avg / total       0.37      0.29      0.32      4021

